## Capstone Project - Different places -Different customers- Toronto 
### Final week 5
### Author : Umeshpratap Singh
#### This project is final journey towards IBM -Data Science Certification
It is a capstone proejct to demonstrate my learing in various data science subjects using python learned over course of IBM data science certification


### Part 1: A description of the problem and a discussion of the background
#### Back ground and Problem set : 
##### •	A marketing research firm wants to know more about booming real estate industry (in commercial /shop) of Canada’s Financial Capital Toronto . It wants to highlight key business activities and preferred locations of shops based on customer preference to its clients.  
##### •	further, it has short listed Main Toronto as location to be drilled down  but are not sure which part of Main Toronto, it should suggest as preffered location to its clients  and which type of shop/commercial category  it should advise to its client . 
### Solution logic : 
##### •	Typically , an area where most of shopping / commercial outlets are located , suggests that it has more commercial activities in and around that area. 
##### •	Further, based on simliar grouping, we can identify customer specific locations i.e what type of customer visit that place and/or what activity they perform. 
#### Approach : 
##### •	We can demarcate main Toronto area on the basis of pin codes and then using Foursquare API can identify which area has maximum stores/venues located within 500 meters of radius. 
##### •	Further, with using clustering algorithm, we can divide each loaction based on type of venues /shops present , indicating what type of customers /activity is preferred in each locations



### Target Audience :
#####  Clients of Market research firm : Any investor who wants to invest in commercial real estate property in Toronto and/or firms  finding good location to establish their shopping unit based on customer preference of that location. 


### Part II Data Requirment : 
##### We will require mainly two data sets. 
    
 ##### data set require for downloading pin code wise demarcation of Toronto city. 
##### •	For this we will be using url form Wikipedia https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  and will do web scrapping 

##### •	further , we will required  latitude and longitude of each neighborhood,   for that we will get data from http://cocl.us/Geospatial_data so we will further refine the data set to reduce the number of columns .


### 2. Foursquare API data : for category selection, feedback and popularity index , we will need Foursquare API to get required data and map them on Mumbai data set.
Api which we will be using are as follows 

#### •	venue search : 
######    o	Request : GET https://api.foursquare.com/v2/venues/search
######    o	Returns a list of venues near the current location, optionally matching a search term.
#### •	Explore a place : 
######    o	Request : https://api.foursquare.com/v2/venues/explore?
######    o	Returns total places near by 



    

In [1]:
import pandas as pd
import numpy as np


##  Data Wrangling from Wikipedia for Toronto 

In [2]:
#  install BeautiulSoup  
 ! pip install requests BeautifulSoup4  # only once


In [2]:
# import library 
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [3]:
# function to get URL and point out issue if any 
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    


In [4]:
# creating object
raw_html = simple_get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
len(raw_html)


56520

In [5]:
# using beautifulSoup to convert web site table in to dataframe
soup = BeautifulSoup(raw_html, 'html.parser')

table = soup.find('table', attrs={'class':'wikitable sortable'}) # required table on webiste
table_rows = table.find_all('tr')


res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


df_toronto = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighborhood"])

df_toronto.head()



,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
# removing not assigined value in Borough
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']
df_toronto.head(5)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
# to check how many Neighbourhood has not assigned value 
dftest = df_toronto[df_toronto.Neighborhood == 'Not assigned']
dftest.head()

,PostalCode,Borough,Neighborhood


#### There are no Neighbourhood which are not assigned 

In [8]:
df_toronto.groupby(['PostalCode']).count()

,Borough,Neighborhood
PostalCode,,
M1B,1,1
M1C,1,1
M1E,1,1
M1G,1,1
M1H,1,1
M1J,1,1
M1K,1,1
M1L,1,1
M1M,1,1


#### there are no postal code which is repeated twice and hence no merging of rows requried 

In [9]:
df_toronto.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df_toronto.shape # final shape of Data frame 

(103, 3)

###  Finding lat and long of each postal code

In [11]:
# Download file 
!wget -q -O 'toronto_latlon.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [12]:
latlan_df = pd.read_csv('toronto_latlon.csv')
latlan_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging postal codes with lat and long table using pd.merge 

In [13]:
df_ftoronto = pd.merge(df_toronto, latlan_df, left_on='PostalCode', right_on='Postal Code')
df_ftoronto.drop(["Postal Code"], axis = 1, inplace = True) # drop the second postal code

df_ftoronto.head() # final toronto DF 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Now, we have created required Dataframe of Toronto postal code with all latttiude and longitude details. 
#### Now we will use Foursquare API and Folium to locate most popular store in most dense area. 

### Imoprting folium and other required libaraies to  map on Toronto data  with foursquare Api

In [14]:
#import required libiaries 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
##!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


     |████████████████████████████████| 102kB 10.1MB/s ta 0:00:01
Libraries imported.


In [15]:
# find lat long for Toronto , to map 
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Creating map of Toronto

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_ftoronto['Latitude'], df_ftoronto['Longitude'], df_ftoronto['Borough'], df_ftoronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Zooming in Downtown Toronto to locate all sotres 

In [17]:
#  Main town toronto

main_toronto_data = df_ftoronto[df_ftoronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
print (main_toronto_data.shape)
main_toronto_data.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [18]:
address = 'Main Toronto, TO' # Zooming in 

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6534817, -79.3839347.


In [19]:
# create map of Main Toronto using latitude and longitude values
map_main_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(main_toronto_data['Latitude'], main_toronto_data['Longitude'], main_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_main_toronto)  
    
map_main_toronto

In [20]:
#my credential
CLIENT_ID = 'LDBN1VLX5GUKN31ABNLDPRLDYGQG0ZFR51AVIYI2QQOVJQMO' # your Foursquare ID
CLIENT_SECRET = 'MS4UUFERGH5VCOQUARZEJWA4TVWPI51NRN2BTUYMRVD4TMOX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LDBN1VLX5GUKN31ABNLDPRLDYGQG0ZFR51AVIYI2QQOVJQMO
CLIENT_SECRET:MS4UUFERGH5VCOQUARZEJWA4TVWPI51NRN2BTUYMRVD4TMOX


#### Getting 100 venues near 

In [21]:
LIMIT = 100# limit of number of venues returned by Foursquare API

radius = 3500 # we need 3.5 km area to scan

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=LDBN1VLX5GUKN31ABNLDPRLDYGQG0ZFR51AVIYI2QQOVJQMO&client_secret=MS4UUFERGH5VCOQUARZEJWA4TVWPI51NRN2BTUYMRVD4TMOX&v=20180605&ll=43.6534817,-79.3839347&radius=3500&limit=100'

In [22]:
# checking with four square and getting the list 
results = requests.get(url).json()


In [23]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

### borrowing function from foursquare

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5227bb01498e17bf485e6202-0,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",5227bb01498e17bf485e6202,NaN,CA,Toronto,Canada,NaN,113,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65323167517444...",43.653232,-79.385296,NaN,NaN,ON,Downtown Toronto,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ad4c05ef964a520a6f620e3-1,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",4ad4c05ef964a520a6f620e3,100 Queen St W,CA,Toronto,Canada,at Bay St,138,"[100 Queen St W (at Bay St), Toronto ON M5H 2N...","[{'label': 'display', 'lat': 43.65227047322295...",43.652270,-79.383516,NaN,M5H 2N1,ON,Nathan Phillips Square,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-57eda381498ebe0e6ef40972-2,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",57eda381498ebe0e6ef40972,220 Yonge St,CA,Toronto,Canada,at Dundas St W,378,"[220 Yonge St (at Dundas St W), Toronto ON M5B...","[{'label': 'display', 'lat': 43.65591027779457...",43.655910,-79.380641,Downtown Toronto,M5B 2H1,ON,UNIQLO ユニクロ,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ad4c062f964a520b5f720e3-3,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",4ad4c062f964a520b5f720e3,189 Yonge St,CA,Toronto,Canada,btwn Queen St E & Shuter St,437,"[189 Yonge St (btwn Queen St E & Shuter St), T...","[{'label': 'display', 'lat': 43.65339379601958...",43.653394,-79.378507,NaN,M5B 2H1,ON,Elgin And Winter Garden Theatres,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-506db1a9e4b0a3f3b31412f0-4,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",506db1a9e4b0a3f3b31412f0,1 Richmond Street West,CA,Toronto,Canada,Yonge Street,432,"[1 Richmond Street West (Yonge Street), Toront...","[{'label': 'display', 'lat': 43.65156872562108...",43.651569,-79.379266,NaN,NaN,ON,Richmond Station,0,[],NaN


#### Cleaning the json and creating dataframe 

In [26]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Downtown Toronto,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.653232,-79.385296
1,Nathan Phillips Square,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",43.652270,-79.383516
2,UNIQLO ユニクロ,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",43.655910,-79.380641
3,Elgin And Winter Garden Theatres,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",43.653394,-79.378507
4,Richmond Station,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",43.651569,-79.379266


In [102]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
3,Elgin And Winter Garden Theatres,Theater,43.653394,-79.378507
4,Richmond Station,American Restaurant,43.651569,-79.379266


In [103]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### finding number of venues in each postal code area of Downtown_Toronto 

We will create a function to call foursquare api for multiple time for each of postal code and append the relevant data into our orignal downtown_venues dataframe

In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [105]:
main_toronto_venues = getNearbyVenues(names=main_toronto_data['Neighborhood'],
                                   latitudes=main_toronto_data['Latitude'],
                                   longitudes=main_toronto_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [107]:
print (main_toronto_venues.shape)
main_toronto_venues.head()

(1647, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


### Finding number of venues on eahc postal code neighborhood

In [108]:
main_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,67,67,67,67,67,67
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


### Analysing each Neigbhorhood 

In [110]:
# one hot encoding
main_toronto_onehot = pd.get_dummies(main_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
main_toronto_onehot['Neighborhood'] = main_toronto_venues['Neighborhood'] 

main_toronto_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [111]:
# moving neighborhood column to first
col_name="Neighborhood"
first_col = main_toronto_onehot.pop(col_name)
main_toronto_onehot.insert(0, col_name, first_col)
print (main_toronto_onehot.shape)
main_toronto_onehot.head()

(1647, 234)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0

#### Next, join group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [151]:
main_toronto_grouped = main_toronto_onehot.groupby('Neighborhood').mean().reset_index()
print (main_toronto_grouped.shape)

main_toronto_grouped

(39, 234)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.0000,0.017544,0.035088,0.000000,0.000000,0.00,0.017544,0.017544,0.000000,0.035088,0.000000,0.000000,0.017544,0.000000,0.0000,0.000000,0.0000,0.00,0.017544,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.035088,0.000000,0.000000,0.0000,0.000000,0.035088,0.000000,0.000000,0.000000,0.00,0.000000,0.035088,0.105263,0.000000,0.000000,0.000000,0.00,0.00,0.017544,0.000000,0.017544,0.000000,0.000000,0.000,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.0

#### finding the top 5 most common venues of each Neigbhorhood 

In [173]:
num_top_venues = 4

for hood in main_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = main_toronto_grouped[main_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 6})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue      freq
0     Coffee Shop  0.105263
1    Cocktail Bar  0.035088
2     Cheese Shop  0.035088
3  Farmers Market  0.035088


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.12
1  Yoga Studio  0.08
2       Bakery  0.08
3  Coffee Shop  0.08


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue      freq
0    Light Rail Station  0.105263
1  Gym / Fitness Center  0.052632
2               Brewery  0.052632
3                   Spa  0.052632


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue    freq
0    Airport Lounge  0.1250
1   Airport Service  0.1250
2  Airport Terminal  0.1250
3             Plane  0.0625


----Central Bay Street----
                venue      freq
0         Coffee Shop  0.164179
1                Café  0.059701
2      Sandwich Place  0.059701
3

### Sorting data based on venues and Neighbhorhood and putting them in dataframe 

In [149]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [174]:
num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = main_toronto_grouped['Neighborhood']

for ind in np.arange(main_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(main_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(38)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Bakery,Cocktail Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Yoga Studio,Bakery,Breakfast Spot
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant
5,Christie,Grocery Store,Café,Park,Restaurant
6,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel
8,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant
9,Davisville North,Gym,Hotel,Breakfast Spot,Food & Drink Shop


## 4. Cluster Neighborhoods

In [175]:
# set number of clusters
kclusters = 7

main_toronto_grouped_clustering = main_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(main_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:38] 

array([0, 3, 3, 5, 0, 3, 0, 0, 0, 3, 3, 0, 2, 3, 0, 3, 0, 3, 1, 3, 2, 3,
       3, 0, 0, 3, 2, 6, 0, 3, 0, 0, 3, 0, 0, 4, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [176]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

main_toronto_merged = main_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
main_toronto_merged = main_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

main_toronto_merged.head(38) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,College Auditorium,Beer Bar
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Clothing Store,Coffee Shop,Café,Cosmetics Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Café,Coffee Shop,Clothing Store,Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Health Food Store,Pub,Doner Restaurant
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Bakery,Cocktail Bar
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3,Grocery Store,Café,Park,Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,3,Coffee Shop,Café,Restaurant,Bar
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,3,Bakery,Pharmacy,Brewery,Supermarket


In [177]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(main_toronto_merged['Latitude'], main_toronto_merged['Longitude'], main_toronto_merged['Neighborhood'], main_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each cluster 

In [178]:
main_toronto_merged.loc[main_toronto_merged['Cluster Labels'] == 0, main_toronto_merged.columns[[1] + list(range(5, main_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park
1,Downtown Toronto,0,Coffee Shop,Diner,College Auditorium,Beer Bar
5,Downtown Toronto,0,Coffee Shop,Restaurant,Bakery,Cocktail Bar
6,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant
10,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café
12,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store
13,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant
15,East Toronto,0,Park,Brewery,Burrito Place,Ice Cream Shop
16,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel
24,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Liquor Store


### first cluster has lots of meeting joints  like coffee shops , cafe , restaurant and hotel. this indicate that this cluster has lots of commercial activity in a day and has lots of office. If some one wants to target office goers can open store in this cluster. We name it "Office_center"

In [179]:
main_toronto_merged.loc[main_toronto_merged['Cluster Labels'] == 1, main_toronto_merged.columns[[1] + list(range(5, main_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
18,Central Toronto,1,Dim Sum Restaurant,Park,Swim School,Bus Line


### not much is captured here , except it has swim school , we can make this cluster non meaningful cluster

In [180]:
main_toronto_merged.loc[main_toronto_merged['Cluster Labels'] == 2, main_toronto_merged.columns[[1] + list(range(5, main_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
21,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant
29,Central Toronto,2,Restaurant,Park,Trail,Deli / Bodega
33,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio


#### This cluster clearly suggests that it has lots of open spaces and is ideal for adventure sports shops complementing trail and park already present in each location . We can name this cluster as * "Open Space"*

In [182]:
main_toronto_merged.loc[main_toronto_merged['Cluster Labels'] == 3, main_toronto_merged.columns[[1] + list(range(5, main_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
2,Downtown Toronto,3,Clothing Store,Coffee Shop,Café,Cosmetics Shop
3,Downtown Toronto,3,Café,Coffee Shop,Clothing Store,Restaurant
7,Downtown Toronto,3,Grocery Store,Café,Park,Restaurant
8,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Bar
9,West Toronto,3,Bakery,Pharmacy,Brewery,Supermarket
11,West Toronto,3,Bar,Coffee Shop,Asian Restaurant,Restaurant
14,West Toronto,3,Café,Yoga Studio,Bakery,Breakfast Spot
17,East Toronto,3,Café,Coffee Shop,American Restaurant,Bakery
20,Central Toronto,3,Gym,Hotel,Breakfast Spot,Food & Drink Shop
22,West Toronto,3,Thai Restaurant,Mexican Restaurant,Café,Discount Store


###  this above cluster has cafe , hotel , bar, discount store , clothing store etc . indicating it has mixed customers are with part residential and part commerical space presence

In [183]:
main_toronto_merged.loc[main_toronto_merged['Cluster Labels'] == 4, main_toronto_merged.columns[[1] + list(range(5, main_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
4,East Toronto,4,Trail,Health Food Store,Pub,Doner Restaurant


#### Not much can be identified in the above cluster, we can demarcate it as non meaningful for our purpose

In [184]:
main_toronto_merged.loc[main_toronto_merged['Cluster Labels'] == 5, main_toronto_merged.columns[[1] + list(range(5, main_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
32,Downtown Toronto,5,Airport Lounge,Airport Service,Airport Terminal,Boutique


#### This above cluster clearly indicates the presence of Airport and allied services and anybody intreseted in airport related business activity must prefer this 

In [185]:
main_toronto_merged.loc[main_toronto_merged['Cluster Labels'] == 6, main_toronto_merged.columns[[1] + list(range(5, main_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
19,Central Toronto,6,Home Service,Garden,Music Venue,Yoga Studio


#### The above cluster has home service , garden and yoga studio, clearly indicating that its a residential area and accordingly somebody should plan a shop/outlet in this area

### Summary of Finding 
###### Based on data, we have find out that Main Toronto has following clusters 

#### 1. Office spaces - ideal for joints like coffee shops , restraurent ete 
#### 2. Open spaces - ideal for adeventure sports shopping 
#### 3 . Mixed spaces - has mix of residential and commercial - good for shopping malls with joints 
#### 4 . Airport - good for airport related joints/ services- duty free shops  
##### 5. Residential - good for home service, home decor etc 